<a href="https://colab.research.google.com/github/kavish-24/Konkani_Mentall_Health/blob/main/DataPreparationPrudentMedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install evaluate odfpy pydub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160673 sha256=b890f2c844ddccad730833bfe0fb06ca625e6aa0319835f46a49b8a7a0055d4c
  Stored in directory: /root/.cache/pip/wheels/36/5d/63/8243a7ee78fff0f944d638fd0e66d7278888f5e2285d7346b6
Successfully built odfpy


In [28]:
from odf import text, teletype
from odf.opendocument import OpenDocumentText
from odf.style import Style, TextProperties
from odf.text import P
import os
import re

def extract_text_from_odt(odt_path, skip_bold=True, debug=False):
    """
    Extract text from ODT file, skipping all bold text (including letters, spaces, and punctuation)
    and removing all noise (punctuation, extra spaces) from non-bold text.

    Args:
        odt_path: Path to input ODT file
        skip_bold: If True, skip all bold text
        debug: If True, print debug information

    Returns:
        List of cleaned non-bold text paragraphs
    """
    from odf.opendocument import load

    if not os.path.exists(odt_path):
        raise FileNotFoundError(f"ODT file not found: {odt_path}")

    try:
        doc = load(odt_path)
    except Exception as e:
        raise ValueError(f"Failed to load ODT file: {e}")

    bold_styles = set()

    # Check automatic styles for bold
    for style in doc.automaticstyles.getElementsByType(Style):
        style_name = style.getAttribute('name')
        for prop in style.getElementsByType(TextProperties):
            font_weight = prop.getAttribute('fontweight')
            if font_weight and 'bold' in str(font_weight).lower():
                bold_styles.add(style_name)
                if debug:
                    print(f"   Found bold style: {style_name}")

    # Check named styles for bold
    for style in doc.styles.getElementsByType(Style):
        style_name = style.getAttribute('name')
        for prop in style.getElementsByType(TextProperties):
            font_weight = prop.getAttribute('fontweight')
            if font_weight and 'bold' in str(font_weight).lower():
                bold_styles.add(style_name)
                if debug:
                    print(f"   Found bold style: {style_name}")

    print(f"\n🔍 Detected {len(bold_styles)} bold styles: {bold_styles}")

    extracted_parts = []
    skipped_count = 0
    kept_count = 0
    skipped_text_samples = []
    kept_text_samples = []

    def get_style_name(node):
        """Get style name from a node using multiple methods"""
        # Try different attribute names
        for attr_name in ['stylename', 'style-name']:
            try:
                style = node.getAttribute(attr_name)
                if style:
                    return style
            except:
                pass

        # Try namespace-aware retrieval
        try:
            style = node.getAttrNS(
                "urn:oasis:names:tc:opendocument:xmlns:text:1.0",
                "style-name"
            )
            if style:
                return style
        except:
            pass

        return None

    def is_node_bold(node):
        """Check if a node has bold styling"""
        style_name = get_style_name(node)
        if style_name and style_name in bold_styles:
            return True
        return False

    def process_node(node, parent_is_bold=False):
        """Recursively process a node and its children"""
        nonlocal skipped_count, kept_count, skipped_text_samples, kept_text_samples

        result_text = ""

        # Check if current node is bold
        current_is_bold = parent_is_bold or is_node_bold(node)

        if node.nodeType == node.TEXT_NODE:
            node_text = node.data
            if node_text.strip():
                if skip_bold and current_is_bold:
                    skipped_count += 1
                    if len(skipped_text_samples) < 5:
                        skipped_text_samples.append(node_text.strip()[:50])
                    if debug:
                        print(f"   [SKIP] TEXT_NODE (bold): {repr(node_text.strip()[:50])}")
                else:
                    result_text += node_text
                    kept_count += 1
                    if len(kept_text_samples) < 5:
                        kept_text_samples.append(node_text.strip()[:50])
                    if debug:
                        print(f"   [KEEP] TEXT_NODE: {repr(node_text.strip()[:50])}")

        elif node.nodeType == node.ELEMENT_NODE:
            # For span elements, check if they're bold
            if node.tagName == "text:span":
                span_is_bold = current_is_bold or is_node_bold(node)

                # Get all text from this span (including nested elements)
                span_full_text = teletype.extractText(node)

                if skip_bold and span_is_bold:
                    skipped_count += 1
                    if span_full_text.strip() and len(skipped_text_samples) < 5:
                        skipped_text_samples.append(span_full_text.strip()[:50])
                    if debug:
                        style_name = get_style_name(node)
                        print(f"   [SKIP] SPAN (bold - {style_name}): {repr(span_full_text[:50])}")
                    # Don't process children if parent span is bold
                    return ""
                else:
                    if debug and span_full_text.strip():
                        style_name = get_style_name(node)
                        print(f"   [KEEP] SPAN ({style_name}): {repr(span_full_text[:50])}")
                    # Process children with current bold status
                    for child in node.childNodes:
                        result_text += process_node(child, span_is_bold)
            else:
                # For other elements, process children
                for child in node.childNodes:
                    result_text += process_node(child, current_is_bold)

        return result_text

    for paragraph in doc.getElementsByType(text.P):
        # Check if paragraph itself is bold
        para_is_bold = is_node_bold(paragraph)

        if debug and para_is_bold:
            print(f"\n⚠️  Paragraph itself is BOLD - will skip all content")

        para_text = ""
        for node in paragraph.childNodes:
            para_text += process_node(node, para_is_bold)

        if para_text.strip():
            # Clean text: remove all punctuation and normalize spaces
            para_text = re.sub(r'\.{2,}', ' ', para_text)  # Replace 2+ dots with single space
            para_text = re.sub(r'[!?,;:"\'()\[\]{}\-—*]+', '', para_text)  # Remove other punctuation

            para_text = re.sub(r'\s+', ' ', para_text)  # Normalize spaces
            para_text = para_text.strip()
            if para_text:
                extracted_parts.append(para_text)

    print(f"\n📊 Extraction summary:")
    print(f"   Kept: {kept_count} elements")
    print(f"   Skipped (bold): {skipped_count} elements")
    print(f"   Extracted paragraphs: {len(extracted_parts)}")

    if skipped_count > 0:
        print(f"\n❌ Sample SKIPPED bold text:")
        for sample in skipped_text_samples:
            print(f"   • {repr(sample)}...")

    if kept_count > 0:
        print(f"\n✅ Sample KEPT non-bold text:")
        for sample in kept_text_samples:
            print(f"   • {repr(sample)}...")

    if extracted_parts:
        print(f"\n🧼 Sample CLEANED paragraphs:")
        for i, para in enumerate(extracted_parts[:3]):
            print(f"   • [{i}] {repr(para)[:100]}...")

    if skipped_count == 0 and skip_bold:
        print(f"\n⚠️  WARNING: No bold text was found to skip! Check ODT styles.")

    return extracted_parts

def create_new_odt(output_path, paragraphs):
    """
    Create a new ODT file with the given paragraphs.

    Args:
        output_path: Path to save the new ODT file
        paragraphs: List of text paragraphs to include
    """
    doc = OpenDocumentText()

    for para_text in paragraphs:
        p = P(text=para_text)
        doc.text.addElement(p)

    try:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        doc.save(output_path)
        print(f"\n💾 New ODT file created: {output_path}")
    except Exception as e:
        raise ValueError(f"Failed to save new ODT file: {e}")

def process_directory(input_dir, output_dir, debug=False):
    """
    Process all .odt files in a directory and create cleaned files
    in the output directory.
    """

    if not os.path.exists(input_dir):
        raise FileNotFoundError(f"Input directory not found: {input_dir}")

    os.makedirs(output_dir, exist_ok=True)

    odt_files = [f for f in os.listdir(input_dir) if f.lower().endswith(".odt")]

    if not odt_files:
        print("❌ No .odt files found in input directory!")
        return

    print(f"📂 Found {len(odt_files)} ODT files to process.\n")

    for file_name in odt_files:
        input_path = os.path.join(input_dir, file_name)
        output_name = file_name.replace(".odt", "_non_bold.odt")
        output_path = os.path.join(output_dir, output_name)

        print(f"➡️ Processing: {file_name}")

        try:
            paragraphs = extract_text_from_odt(input_path, skip_bold=True, debug=debug)
            create_new_odt(output_path, paragraphs)
            print(f"✅ Saved cleaned file: {output_path}\n")

        except Exception as e:
            print(f"❌ Error processing {file_name}: {e}\n")



def main():
    # Set your folders here
    INPUT_DIR = "/content/drive/MyDrive/Anju Project (1)/Transcript/October"
    OUTPUT_DIR = "/content/drive/MyDrive/Anju Project (1)/Cleaned Transcript/Octobercleaned"
    DEBUG = False  # Set True if you want full logs

    print("="*70)
    print("PROCESSING DIRECTORY OF ODT FILES - REMOVING ALL BOLD TEXT")
    print("="*70)

    try:
        process_directory(INPUT_DIR, OUTPUT_DIR, debug=DEBUG)
        print("\n✅✅ ALL FILES PROCESSED SUCCESSFULLY ✅✅")

    except Exception as e:
        print(f"\n❌ Failed: {e}")

if __name__ == "__main__":
    main()

PROCESSING DIRECTORY OF ODT FILES - REMOVING ALL BOLD TEXT
📂 Found 8 ODT files to process.

➡️ Processing: 2 OCT PRIME.odt

🔍 Detected 70 bold styles: {'T13', 'T51', 'T58', 'P69', 'P88', 'T28', 'T87', 'T16', 'T61', 'P70', 'T7', 'T82', 'T100', 'T14', 'T112', 'T42', 'T116', 'P3', 'T75', 'T38', 'T97', 'P19', 'P5', 'T39', 'T102', 'T40', 'P45', 'P21', 'T12', 'P20', 'P10', 'T60', 'T57', 'T104', 'T30', 'P17', 'P2', 'T66', 'T93', 'T62', 'T44', 'T47', 'T25', 'T49', 'T106', 'P1', 'T37', 'T73', 'T79', 'P76', 'T109', 'T34', 'P9', 'P22', 'P4', 'T53', 'T59', 'P18', 'P71', 'T85', 'T114', 'Heading4', 'T95', 'T91', 'P8', 'T55', 'T68', 'T32', 'T36', 'T29'}

📊 Extraction summary:
   Kept: 26 elements
   Skipped (bold): 66 elements
   Extracted paragraphs: 22

❌ Sample SKIPPED bold text:
   • '2 ऑक्टोबर 2017 – प्रायम (पीके, नेने,अमर, अपर्णा)'...
   • 'अँकर – सुयश गावणेकार'...
   • '1.(प्रुडन्ट)'...
   • 'फस्ट ब्रेक'...
   • '2.(फायर)(ब्युरे)'...

✅ Sample KEPT non-bold text:
   • 'नमस्कार पळोवया प्रुडंट ख

In [18]:
import zipfile
from xml.etree import ElementTree as ET

file_path = "/content/drive/MyDrive/Anju Project (1)/Audio Prudent media (1)/August 2017 (1)/dataset/10 AUG PRIME_non_bold.odt"

def read_odt(path):
    with zipfile.ZipFile(path, 'r') as z:
        xml_content = z.read("content.xml")

    root = ET.fromstring(xml_content)

    # ODT text nodes live inside <text:p> and <text:span>
    ns = {"text": "urn:oasis:names:tc:opendocument:xmlns:text:1.0"}

    paragraphs = []
    for p in root.findall(".//text:p", ns):
        text_content = ""
        for node in p.iter():
            if node.text:
                text_content += node.text
        paragraphs.append(text_content.strip())

    return "\n".join(paragraphs)

text_output = read_odt(file_path)
print(text_output)


नमस्कार पळोवया प्रुडंट खबरो
टॅंकरवाल्यांक ना धरबांद गोंयभरच्या वॉटर टॅंकरांचें ट्रान्स्पोर्ट डिपोर्टमेन्ट करतलो सेफ्टीऑडिट उदका पुरवणे उपरांत लोकांकडच्यान सय घेवपाची सिस्टम प्रुडंटाचे ऑपा वॉट टॅंकर एक्स्पोजेचो इम्पॅक्ट.
कळसाभंडुरा प्रकल्पाक केंद्राची परवानगी ना प्रॉजॅक्ट जाला म्हण्टा ती फट म्हादय बचाव अभियानान कर्नाटकाक सुप्रिम कोर्टांत केलें एक्स्पोज कोर्टान मागलें केंद्र कर्नाटकाकडच्यान एफिडॅव्हिट.
एक्टिव्हिस्टांचो कॉंग्रेसीक तेंको रायबंदरा पर्रीकारान कांयच कामां केल्लीं ना लायटउदकाच्यो कटकटी गिरीशाचो रायबंदरा प्रचार पर्रीकार खंय सगळ्यात भ्रष्ट मुख्यमंत्री
आयरीशाक जीतो मारता म्हूण दिल्ले धमकेचे केशींत विश्वजीतान हायकोर्टांत भल्लो देड लाख लीगल एड फंड चार्जशीट रद्द केस बंद 10 वर्सां उपरांत आयरिश म्हण्टा हरकत ना.
मुरगांवचो आमदार मिलींदाची वास्कोसावन बोरयेशिरोडा चलत साईबाबाचे दर्शनाक वचपाची आंगवण मिलिंदाची कार्यकर्त्यांक घेवन फांतोडेरसावनबोरयेमेरेन 7 वरांची वारी
आनी
भारतांत फुटसालाक येतले बरे दिस ए.एफ.सी आनी ए.आय.एफ.एफ मेळून तयार करता फुटसाल डॅव्हलापमॅन्टाचो आराखडो.
गोंयभरच्या वॉटर टॅंकर

In [33]:
import os

# path to your folder containing txt files
folder_path = "/content/drive/MyDrive/Anju Project (1)/Audio Prudent media (1)/transcript segment/10 Aug"

# loop through the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".txt"):  # only .txt files
        file_path = os.path.join(folder_path, file_name)

        # open file with utf-8 encoding
        with open(file_path, 'r', encoding='utf-8') as file:
            print(f"--- {file_name} ---")
            content = file.read()
            print(content)
            print("\n")  # spacing between files


--- Copy of Copy of Konkani Prime News_100817_segment_014_transcript.txt ---
43 मतानी जी किल्लो स्वतंद्रताय दीस्तेकलो आइकपाक याना उलोग कलना अश्या दिव्यांग लोकां खते राता साइन लेंग्वेजिन राष्टर गीत लाउंच जाला ब्राधर्हूड न्यू दिल्ली और डिजाबिलिटी राइटस असोसियेशन गोआ कला अकादमीत बरोबर गहुन साइन लेंग्वेजिन लो नैशनल एन्थम विडियो लाउंच गहलो अभी ता बच्चना गहुन है खास राष्टर गीत के लाउंच इतल्याने बुलाइटी नांगात सब्ता यो खोबरतुं को प्रूड और वाटसेब ट्वीटर


--- Copy of Copy of Konkani Prime News_100817_segment_015_transcript.txt ---
फेस्बूक औनि प्रुडन्ट वेबसाइटी चेरी लाइव मेट आत। प्रुडन्ट आक फ़ोलोग करी थरावाद, पही थरावाद प्रुडन्ट, नमस्कार।


--- Copy of Copy of Konkani Prime News_100817_segment_003_transcript.txt ---
मिलिंदाची कार्ये गर्ध्यां ग्यों फातोडेर सांध बोरिये मिरें साथ वरांची वारी अनि भारतांट फुटसाला गेतले बरे दिस AFC आणि AIFF मेंयों तायार करता फुटसाल डेवलाप्मेंटाचा आराखडव गौइंबर्चा वाटा टेंकराज्या आता ट्रांस्पोर्ट डेपाट्मेंट सिफ्टी ओडिट कुरतला ते पिवपाज्यो उठकाक सेफ आसात काईना अनि त्रि

In [39]:
import re

raw = r'''
file_name,matched_text
Copy of Copy of Konkani Prime News_100817_segment_014_transcript.txt,स्वतंत्रताय दीस तेंकलो आयकपाक येना उलोवंक कळना अश्या दिव्यांग लोकांखातीर आता सायन लॅग्वेजीन राष्ट्रगीत लाँच जालां. ब्रदरहुड न्यु दिल्ली आनी डिसॅबीलीटी रायट्स असोसिएशन ऑफ गोवा कला अकादमीक बरोबर घेवन सायन लॅग्वीजींतलो नॅशनल एन्थम व्हिडियो लाँच केलो. अमिताभ बच्चनाक घेवन हें खास राष्ट्रगीत केलां. हें बुलॅटिन हांगांच सोंपता ह्यो खबरो तुमकां प्रुडंट वॉट्सॅप ट्विटर
Copy of Copy of Konkani Prime News_100817_segment_015_transcript.txt,फेसबूक आनी प्रुडंट व्हॅबसायटीचेर लायव्ह मेळटात प्रुडंटाक फॉलो करात नमस्कार
Copy of Copy of Konkani Prime News_100817_segment_003_transcript.txt,मिलींदाची वास्कोसावन बोरयेशिरोडा चलत साईबाबाचे दर्शनाक वचपाची आंगवण मिलिंदाची कार्यकर्त्यांक घेवन फांतोडेरसावनबोरयेमेरेन 7 वरांची वारी आनी भारतांत फुटसालाक येतले बरे दिस ए.एफ.सी आनी ए.आय.एफ.एफ मेळून तयार करता फुटसाल डॅव्हलापमॅन्टाचो आराखडो. गोंयभरच्या वॉटर टॅंकरांचें आतां ट्रान्स्पोर्ट डिपोर्टमेन्ट सेफ्टी ऑडिट करतलो.ते पिवपाचें उदकाक सेफ आसात काय ना आनी ट्रिपी कॉंत्रादाप्रमाण लोकांमेरेन पावतात काय ना ताचीय खातरजमा जातली.एखाद्या सुवातीर उदका पुरवण
Copy of Copy of Konkani Prime News_100817_segment_002_transcript.txt,पर्रीकार खंय सगळ्यात भ्रष्ट मुख्यमंत्री
Copy of Copy of Konkani Prime News_100817_segment_008_transcript.txt,तशें सर्कुलरुय हे पयली खंय खात्यान काडलां. इलेक्शनाचोकोड सुरु जावचें पयली रिक्रुटमेंट जाला तांच्योच अपॉयंटमेंट ऑर्डरी खंय रिझल्टा उपरांत दिवपाच्यो. कोंबा रिंग रोडावेल्या 15 फामिलींक चवथी उपरांत हालोवचे तांचें हावसींग बोर्डांत पुनर्वसन जावंक जाय म्हूण टिसीपी मंत्री विजय उलयला. थंयचे फामीलींची फातोड्डेच्या आमदारासयत मडगांव कलॅक्टरेटांत बसका जाली.
Copy of Copy of Konkani Prime News_100817_segment_001_transcript.txt,नमस्कार पळोवया प्रुडंट खबरो टॅंकरवाल्यांक ना धरबांद गोंयभरच्या वॉटर टॅंकरांचें ट्रान्स्पोर्ट डिपोर्टमेन्ट करतलो सेफ्टीऑडिट उदका पुरवणे उपरांत लोकांकडच्यान सय घेवपाची सिस्टम प्रुडंटाचे ऑपा वॉट टॅंकर एक्स्पोजेचो इम्पॅक्ट. कळसाभंडुरा प्रकल्पाक केंद्राची परवानगी ना प्रॉजॅक्ट जाला म्हण्टा ती फट म्हादय बचाव अभियानान कर्नाटकाक सुप्रिम कोर्टांत केलें एक्स्पोज
Copy of Copy of Konkani Prime News_100817_segment_009_transcript.txt,बार्देस पेडणेंच्या गोवा कूळ मुंडकार संघर्श समितीन पणजे येवन नवें कूळ मुंडकार दुरुस्ती विधेयक जाळपाचो लान केलो खरो
Copy of Copy of Konkani Prime News_100817_segment_013_transcript.txt,गोंयच्यो साधन सुविदा तांकां मानवल्यात. मुरगांवचो बीजेपी आमदार मिलींद नायकान आंगवण पुराय केली वास्को ते बोरये 28 किलोमिटर तो साई बाबाचें दर्शनाक चलत गेलो. बरोबर धायेक कार्यकर्त्यांचीय वारी मिलींदान सगल्यांक बरोबर घेवन फांतोडेर पाचंक वास्कोसावन बोरयेची वाट धल्ली. इलॅक्शनाकडेन मिलिंद जिकल्यार बोरये वारीची साईबाबाक ताचे कार्य़कर्त्यान आंगवण केल्ली. मिलिंद 143
Copy of Copy of Konkani Prime News_100817_segment_012_transcript.txt,म्हूण शिक्षण संचालक गजाजन भट प्रुडंटाकडेन उलयलो. सिटिझन जर्नलिस्टाचे व्हिडिओवेल्यान प्रुडंटान हो प्रकार एक्स्पोज केल्लो. ऑल इंडिया फुटबॉल फॅडरेशन आतां फुटसाल डेव्हलाप करपाचेर लक्ष घालतलें. ह्या कामांत आतां तांकां एशीयन फुटबॉल कन्फॅडरेशन आदार करतलें. इन्फ्रास्ट्रक्चर कशें आसा कांय पळोवंक सद्या एएफसी आनी एआयएफएफाचें शिश्टमंडळ देशभर भोंवतले. गोंयात ब्रेस्तारा इन्स्पेक्शन जालें.
Copy of Copy of Konkani Prime News_100817_segment_007_transcript.txt,पूर्णविराम दिलो. विश्वजी फंड डिपॉजिटूय केलो. वर्स 2007त विश्वजितान आयरीशाक जितो मारपाची धमकी दिली म्हूण हें प्रकरण गाजिल्लें. आयरिशाक खंय ही केस क्लोज करपाक हरकत ना. शाळांच्या मॅनेजमॅन्टांनी रिटायर स्टाफाच्या पॅन्शनचो बेगोबेग सॅटलमेंट करुंक जाय. ना जाल्यार काद्यान तांका दंड म्हूण शिक्षण संचालक गजाजन भट उलयला. एडमीशनावेळार भुरग्यांचे आर्धारकार्ड करुन घेयात.
Copy of Copy of Konkani Prime News_100817_segment_006_transcript.txt,लोकांचो बरो प्रदिसात मेळटा म्हूण काँग्रेसीचो उमेदवार गिरीश चोडणकार मिडियाकडेन उलयलो. एक्टिव्हिस्ट आयरीश गिरीश आनी रॉयाक तेंको डिक्लॅर करपाक काँग्रेस हावजांत मिडियामुखार पावलो. घेवया एक ब्रेक भलायकी मंत्री विश्वजीत राणेन एक्टिविस्ट आयरिश रॉड्रीग्साक जीतो मारपाची धमकी दिल्ली म्हण्टा ती केस सॅटल. ताचे आड चार्जशीट रद्द. लीगल एड फंडाक विश्वजीताकडच्यान देड लाख फंड वसूल करून हायकोर्टान
Copy of Copy of Konkani Prime News_100817_segment_005_transcript.txt,मात काम जाल्लें ना आनी तें नेटान चल्लां म्हूण अभियानान कोर्टाक क्लियर केलें.कर्नाटकान असो अर्ज केला आनी केंद्रान तांकां परवानगी दिल्या जाल्यार दोगांनीय ती कोर्टाक एफिडॅव्हिटाचेर दाखोवची पडटली. रायबंदराचे एक्टिव्हिस्ट कॉंग्रेसीफाटल्यान. मुख्यमंत्री पर्रीकारान रायबंदरा कांयच कामां केल्लीं ना. रायबंदरा लोकांक लायटउदकाच्यो कटकटी. हिंदुक मसुंडी सारकी ना. आपल्या प्रचार
Copy of Copy of Konkani Prime News_100817_segment_004_transcript.txt,कळसाभंडुरा प्रकल्पाचे काम कर्नाटकान केंद्राची परवानगी नासतना सुरु केलां ही गजाल म्हादय बचाव अभियानान सुप्रिम कोर्टांत एक्पोज केल्या.कळसा बंडुरा प्रकल्पाचे काम जवळ जवळ पुराय जायत आयलां म्हूण कर्नाटकान कोर्टाक सांगलां.
Copy of Copy of Konkani Prime News_100817_segment_010_transcript.txt,मात इलॅक्शन कोडांत परमिशन सारकें नाशिल्ल्यान प्लान फिसकटलो. आशिल्ले सात आठ जाणूच. करंजाळे निदर्शनां करता म्हूण सांगून म्हापालिका पोलीस कलॅक्टराचें परमिशन घेतलें. इलॅक्शन कमिशनाक सांगलेंच ना. निदर्शनां करता म्हूण सांगले परमिशन
Copy of Copy of Konkani Prime News_100817_segment_011_transcript.txt,असले प्रकार बंद जावंक जाय म्हूण गोवा वूमन्स फोरम मागता. फोरमान मडगावां जागृती रॅली काडली. दाबोळे केशव स्मृतीच्या अकरावेच्या भुरग्यांक हायर सेकंडरीचे तीन मजली बिल्डिंगेचे तेर्रासाचेर चडोवन टायल्स नितळ करूंक लायिल्ले प्रकरणाची शिक्षण खात्यान दखल घेतल्या. शिक्षण संचालकान एडीईआय कडच्यान रिपोर्ट मागला. रिपोर्ट आयलो काय हायर सेकंडरीच्या प्रिन्सिपलाक नोटीस वतली
'''

# Pattern for both cases:
pattern = r"""
    (?:---\s*)?                # optional leading ---
    ([\w .-]+?\.txt)           # filename ending in .txt
    (?:\s*---)?,?              # optional --- or comma after filename
    ["“]?                      # optional opening quote
    (.+?)                      # the text content
    ["”]?                      # optional closing quote
    (?=\n|$)                   # end at newline
"""

matches = re.findall(pattern, raw, flags=re.DOTALL | re.VERBOSE)

print("{")
for file, text in matches:
    text = text.strip().replace('"', '\\"')

    # Remove header if file contains "file_name"
    if "file_name" in file.lower():
        continue

    print(f'    "{file}": "{text}",')
print("}")


{
    "News_100817_segment_014_transcript.txt": "स्वतंत्रताय दीस तेंकलो आयकपाक येना उलोवंक कळना अश्या दिव्यांग लोकांखातीर आता सायन लॅग्वेजीन राष्ट्रगीत लाँच जालां. ब्रदरहुड न्यु दिल्ली आनी डिसॅबीलीटी रायट्स असोसिएशन ऑफ गोवा कला अकादमीक बरोबर घेवन सायन लॅग्वीजींतलो नॅशनल एन्थम व्हिडियो लाँच केलो. अमिताभ बच्चनाक घेवन हें खास राष्ट्रगीत केलां. हें बुलॅटिन हांगांच सोंपता ह्यो खबरो तुमकां प्रुडंट वॉट्सॅप ट्विटर",
    "News_100817_segment_015_transcript.txt": "फेसबूक आनी प्रुडंट व्हॅबसायटीचेर लायव्ह मेळटात प्रुडंटाक फॉलो करात नमस्कार",
    "News_100817_segment_003_transcript.txt": "मिलींदाची वास्कोसावन बोरयेशिरोडा चलत साईबाबाचे दर्शनाक वचपाची आंगवण मिलिंदाची कार्यकर्त्यांक घेवन फांतोडेरसावनबोरयेमेरेन 7 वरांची वारी आनी भारतांत फुटसालाक येतले बरे दिस ए.एफ.सी आनी ए.आय.एफ.एफ मेळून तयार करता फुटसाल डॅव्हलापमॅन्टाचो आराखडो. गोंयभरच्या वॉटर टॅंकरांचें आतां ट्रान्स्पोर्ट डिपोर्टमेन्ट सेफ्टी ऑडिट करतलो.ते पिवपाचें उदकाक सेफ आसात काय ना आनी ट्रिपी कॉंत्रादाप्रमाण लोकांमेरेन पावतात काय ना ताचीय खातरजमा

In [42]:
import os

# Directory where files will be created
output_dir = "/content/drive/MyDrive/training/10 Aug"

# Create folder if not exists
os.makedirs(output_dir, exist_ok=True)

# Mapping of file_name → matched_text
segments = {
    "News_100817_segment_014_transcript.txt": "स्वतंत्रताय दीस तेंकलो आयकपाक येना उलोवंक कळना अश्या दिव्यांग लोकांखातीर आता सायन लॅग्वेजीन राष्ट्रगीत लाँच जालां. ब्रदरहुड न्यु दिल्ली आनी डिसॅबीलीटी रायट्स असोसिएशन ऑफ गोवा कला अकादमीक बरोबर घेवन सायन लॅग्वीजींतलो नॅशनल एन्थम व्हिडियो लाँच केलो. अमिताभ बच्चनाक घेवन हें खास राष्ट्रगीत केलां. हें बुलॅटिन हांगांच सोंपता ह्यो खबरो तुमकां प्रुडंट वॉट्सॅप ट्विटर",
    "News_100817_segment_015_transcript.txt": "फेसबूक आनी प्रुडंट व्हॅबसायटीचेर लायव्ह मेळटात प्रुडंटाक फॉलो करात नमस्कार",
    "News_100817_segment_003_transcript.txt": "मिलींदाची वास्कोसावन बोरयेशिरोडा चलत साईबाबाचे दर्शनाक वचपाची आंगवण मिलिंदाची कार्यकर्त्यांक घेवन फांतोडेरसावनबोरयेमेरेन 7 वरांची वारी आनी भारतांत फुटसालाक येतले बरे दिस ए.एफ.सी आनी ए.आय.एफ.एफ मेळून तयार करता फुटसाल डॅव्हलापमॅन्टाचो आराखडो. गोंयभरच्या वॉटर टॅंकरांचें आतां ट्रान्स्पोर्ट डिपोर्टमेन्ट सेफ्टी ऑडिट करतलो.ते पिवपाचें उदकाक सेफ आसात काय ना आनी ट्रिपी कॉंत्रादाप्रमाण लोकांमेरेन पावतात काय ना ताचीय खातरजमा जातली.एखाद्या सुवातीर उदका पुरवण",
    "News_100817_segment_002_transcript.txt": "पर्रीकार खंय सगळ्यात भ्रष्ट मुख्यमंत्री",
    "News_100817_segment_008_transcript.txt": "तशें सर्कुलरुय हे पयली खंय खात्यान काडलां. इलेक्शनाचोकोड सुरु जावचें पयली रिक्रुटमेंट जाला तांच्योच अपॉयंटमेंट ऑर्डरी खंय रिझल्टा उपरांत दिवपाच्यो. कोंबा रिंग रोडावेल्या 15 फामिलींक चवथी उपरांत हालोवचे तांचें हावसींग बोर्डांत पुनर्वसन जावंक जाय म्हूण टिसीपी मंत्री विजय उलयला. थंयचे फामीलींची फातोड्डेच्या आमदारासयत मडगांव कलॅक्टरेटांत बसका जाली.",
    "News_100817_segment_001_transcript.txt": "नमस्कार पळोवया प्रुडंट खबरो टॅंकरवाल्यांक ना धरबांद गोंयभरच्या वॉटर टॅंकरांचें ट्रान्स्पोर्ट डिपोर्टमेन्ट करतलो सेफ्टीऑडिट उदका पुरवणे उपरांत लोकांकडच्यान सय घेवपाची सिस्टम प्रुडंटाचे ऑपा वॉट टॅंकर एक्स्पोजेचो इम्पॅक्ट. कळसाभंडुरा प्रकल्पाक केंद्राची परवानगी ना प्रॉजॅक्ट जाला म्हण्टा ती फट म्हादय बचाव अभियानान कर्नाटकाक सुप्रिम कोर्टांत केलें एक्स्पोज",
    "News_100817_segment_009_transcript.txt": "बार्देस पेडणेंच्या गोवा कूळ मुंडकार संघर्श समितीन पणजे येवन नवें कूळ मुंडकार दुरुस्ती विधेयक जाळपाचो लान केलो खरो",
    "News_100817_segment_013_transcript.txt": "गोंयच्यो साधन सुविदा तांकां मानवल्यात. मुरगांवचो बीजेपी आमदार मिलींद नायकान आंगवण पुराय केली वास्को ते बोरये 28 किलोमिटर तो साई बाबाचें दर्शनाक चलत गेलो. बरोबर धायेक कार्यकर्त्यांचीय वारी मिलींदान सगल्यांक बरोबर घेवन फांतोडेर पाचंक वास्कोसावन बोरयेची वाट धल्ली. इलॅक्शनाकडेन मिलिंद जिकल्यार बोरये वारीची साईबाबाक ताचे कार्य़कर्त्यान आंगवण केल्ली. मिलिंद 143",
    "News_100817_segment_012_transcript.txt": "म्हूण शिक्षण संचालक गजाजन भट प्रुडंटाकडेन उलयलो. सिटिझन जर्नलिस्टाचे व्हिडिओवेल्यान प्रुडंटान हो प्रकार एक्स्पोज केल्लो. ऑल इंडिया फुटबॉल फॅडरेशन आतां फुटसाल डेव्हलाप करपाचेर लक्ष घालतलें. ह्या कामांत आतां तांकां एशीयन फुटबॉल कन्फॅडरेशन आदार करतलें. इन्फ्रास्ट्रक्चर कशें आसा कांय पळोवंक सद्या एएफसी आनी एआयएफएफाचें शिश्टमंडळ देशभर भोंवतले. गोंयात ब्रेस्तारा इन्स्पेक्शन जालें.",
    "News_100817_segment_007_transcript.txt": "पूर्णविराम दिलो. विश्वजी फंड डिपॉजिटूय केलो. वर्स 2007त विश्वजितान आयरीशाक जितो मारपाची धमकी दिली म्हूण हें प्रकरण गाजिल्लें. आयरिशाक खंय ही केस क्लोज करपाक हरकत ना. शाळांच्या मॅनेजमॅन्टांनी रिटायर स्टाफाच्या पॅन्शनचो बेगोबेग सॅटलमेंट करुंक जाय. ना जाल्यार काद्यान तांका दंड म्हूण शिक्षण संचालक गजाजन भट उलयला. एडमीशनावेळार भुरग्यांचे आर्धारकार्ड करुन घेयात.",
    "News_100817_segment_006_transcript.txt": "लोकांचो बरो प्रदिसात मेळटा म्हूण काँग्रेसीचो उमेदवार गिरीश चोडणकार मिडियाकडेन उलयलो. एक्टिव्हिस्ट आयरीश गिरीश आनी रॉयाक तेंको डिक्लॅर करपाक काँग्रेस हावजांत मिडियामुखार पावलो. घेवया एक ब्रेक भलायकी मंत्री विश्वजीत राणेन एक्टिविस्ट आयरिश रॉड्रीग्साक जीतो मारपाची धमकी दिल्ली म्हण्टा ती केस सॅटल. ताचे आड चार्जशीट रद्द. लीगल एड फंडाक विश्वजीताकडच्यान देड लाख फंड वसूल करून हायकोर्टान",
    "News_100817_segment_005_transcript.txt": "मात काम जाल्लें ना आनी तें नेटान चल्लां म्हूण अभियानान कोर्टाक क्लियर केलें.कर्नाटकान असो अर्ज केला आनी केंद्रान तांकां परवानगी दिल्या जाल्यार दोगांनीय ती कोर्टाक एफिडॅव्हिटाचेर दाखोवची पडटली. रायबंदराचे एक्टिव्हिस्ट कॉंग्रेसीफाटल्यान. मुख्यमंत्री पर्रीकारान रायबंदरा कांयच कामां केल्लीं ना. रायबंदरा लोकांक लायटउदकाच्यो कटकटी. हिंदुक मसुंडी सारकी ना. आपल्या प्रचार",
    "News_100817_segment_004_transcript.txt": "कळसाभंडुरा प्रकल्पाचे काम कर्नाटकान केंद्राची परवानगी नासतना सुरु केलां ही गजाल म्हादय बचाव अभियानान सुप्रिम कोर्टांत एक्पोज केल्या.कळसा बंडुरा प्रकल्पाचे काम जवळ जवळ पुराय जायत आयलां म्हूण कर्नाटकान कोर्टाक सांगलां.",
    "News_100817_segment_010_transcript.txt": "मात इलॅक्शन कोडांत परमिशन सारकें नाशिल्ल्यान प्लान फिसकटलो. आशिल्ले सात आठ जाणूच. करंजाळे निदर्शनां करता म्हूण सांगून म्हापालिका पोलीस कलॅक्टराचें परमिशन घेतलें. इलॅक्शन कमिशनाक सांगलेंच ना. निदर्शनां करता म्हूण सांगले परमिशन",
    "News_100817_segment_011_transcript.txt": "असले प्रकार बंद जावंक जाय म्हूण गोवा वूमन्स फोरम मागता. फोरमान मडगावां जागृती रॅली काडली. दाबोळे केशव स्मृतीच्या अकरावेच्या भुरग्यांक हायर सेकंडरीचे तीन मजली बिल्डिंगेचे तेर्रासाचेर चडोवन टायल्स नितळ करूंक लायिल्ले प्रकरणाची शिक्षण खात्यान दखल घेतल्या. शिक्षण संचालकान एडीईआय कडच्यान रिपोर्ट मागला. रिपोर्ट आयलो काय हायर सेकंडरीच्या प्रिन्सिपलाक नोटीस वतली",
}

for file_name, text in segments.items():
    file_path = os.path.join(output_dir, file_name)

    with open(file_path, "w", encoding="utf-8") as f:
        f.write(text)

print(f"✅ Created {len(segments)} files in folder: {output_dir}")


✅ Created 15 files in folder: /content/drive/MyDrive/training/10 Aug
